# Packages & data

In [2]:
import pandas as pd
ad_clicks = pd.read_csv("ad_click.csv")

## Analyzing Ad Sources

Examine the first few rows of ad_clicks.

In [3]:
ad_clicks.head(10)

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A
8,018cea61-19ea-4119-895b-1a4309ccb148,email,1 - Monday,18:33,A
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B


How many views (i.e., rows of the table) came from each utm_source?

In [5]:
click_from_sources = ad_clicks.groupby('utm_source').user_id.count().reset_index()
print(click_from_sources)

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [44]:
ad_clicks['is_click'] = ad_clicks.apply(lambda row: 'True' if pd.notna(row['ad_click_timestamp']) else 'False' , axis=1)
# ad_clicks['is_click'] = ad_clicks['ad_click_timestamp'].notna()


We want to know the percent of people who clicked on ads from each utm_source.

In [45]:
clicks_by_source= ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()
print(clicks_by_source)

  utm_source is_click  user_id
0      email    False      175
1      email     True       80
2   facebook    False      324
3   facebook     True      180
4     google    False      441
5     google     True      239
6    twitter    False      149
7    twitter     True       66


Pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.

In [48]:
clicks_pivot = clicks_by_source.pivot(
    columns='is_click',
    index='utm_source',
    values = 'user_id'
).reset_index()


is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


In [55]:
clicks_pivot['percent_clicked'] = (round(clicks_pivot['True']/(clicks_pivot['True']+clicks_pivot['False']),3))*100
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80             31.4
1          facebook    324   180             35.7
2            google    441   239             35.1
3           twitter    149    66             30.7


## Analyzing an A/B Test


In [65]:

#Groupby method
ad_clicks_by_group= ad_clicks.groupby(['experimental_group','is_click']).user_id.count().reset_index().pivot(
    columns= 'is_click',
    index = 'experimental_group',
    values='user_id'
).reset_index()
print(ad_clicks_by_group)



is_click experimental_group  False  True
0                         A    517   310
1                         B    572   255


The clicks might have changed by day of the week
Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

In [69]:
a_clicks = ad_clicks[ad_clicks['experimental_group'] == "A"]
b_clicks = ad_clicks[ad_clicks['experimental_group'] == "B"]


<class 'pandas.core.frame.DataFrame'>


Calculate the percent of users who clicked on the ad by day

### Group A

In [79]:
a_clicks_by_days = a_clicks.groupby(['day','is_click']).user_id.count().reset_index().pivot(
    columns='is_click',
    index='day',
    values='user_id'
).reset_index()
a_clicks_by_days['Percentage'] = (a_clicks_by_days['True']/(a_clicks_by_days['True']+a_clicks_by_days['False']))*100
print(a_clicks_by_days)

is_click            day  False  True  Percentage
0            1 - Monday     70    43   38.053097
1           2 - Tuesday     76    43   36.134454
2         3 - Wednesday     86    38   30.645161
3          4 - Thursday     69    47   40.517241
4            5 - Friday     77    51   39.843750
5          6 - Saturday     73    45   38.135593
6            7 - Sunday     66    43   39.449541


: 

### Group B

In [78]:
b_clicks_by_days = b_clicks.groupby(['day','is_click']).user_id.count().reset_index().pivot(
    columns='is_click',
    index='day',
    values='user_id'
).reset_index()
b_clicks_by_days['Percentage'] = (b_clicks_by_days['True']/(b_clicks_by_days['True']+b_clicks_by_days['False']))*100
print(b_clicks_by_days)

is_click            day  False  True  Percentage
0            1 - Monday     81    32   28.318584
1           2 - Tuesday     74    45   37.815126
2         3 - Wednesday     89    35   28.225806
3          4 - Thursday     87    29   25.000000
4            5 - Friday     90    38   29.687500
5          6 - Saturday     76    42   35.593220
6            7 - Sunday     75    34   31.192661


### So i will recommend my Company to use Group A's Ads :Vv